## タレント一覧ページから人物の情報を取得

In [2]:
import requests
from bs4 import BeautifulSoup

# タレント一覧を掲載しているページを取得
url = "http://geinoupro.com/profil.html"
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')

In [3]:
import re
import datetime
from datetime import date

# 生年月日から年齢を計算
def calculate_age(date):
    today = date.today()
    age = today.year - date.year

    # 今年の誕生日を迎えていなければ、ageを1つ減らす
    # 今日を表すタプル(7, 29) < 誕生日を表すタプル(7, 30)
    if (today.month, today.day) < (date.month, date.day):
        age -= 1
    return age

# ページからタレント情報を取り出す
talent_elements = soup.select("#main-contents ul.news.m15-b li")[1:]
talents = []
for element in talent_elements:
    name = element.select('span')[0].text
    r = re.compile('(.*)（(.*)）')
    m = r.search(name)
    name_kanji = m.group(1).replace('　', ' ')
    name_kana = m.group(2).replace('・', ' ')
    # 生年月日
    r = re.compile('【生年月日】(.*)')
    m = r.search(element.text)
    birthday_string = m.group(1)
    birth_date = datetime.datetime.strptime(birthday_string, '%Y/%m/%d').date()
    talent = dict(
        name_kanji=name_kanji,
        name_kana=name_kana,
        birth_date=birth_date.strftime('%Y-%m-%d'),
        age=calculate_age(birth_date)
    )
    talents.append(talent)

## Bingで画像を検索して、結果画像をダウンロード

In [4]:
import os
import shutil
import urllib
import glob

BING_SEARCH_API_KEY = 'ceaddaf5f4e44ffebfc2dcf639f3f130'

# Bingから画像を検索する関数
def search_image(keyword, count=30):
    url = 'https://api.cognitive.microsoft.com/bing/v7.0/images/search'
    params = {'q': keyword, 'mkt': 'ja-JP', 'count': count}
    headers = {'Ocp-Apim-Subscription-Key': BING_SEARCH_API_KEY}
    res = requests.get(url, params=params, headers=headers)
    return res.json()['value']

# 画像ダウンロード関数
def save_image(url, name, index):
    try:
        response = requests.get(url, stream=True)
    except Exception:
        # ネットワークエラー
        return

    extension = response.headers['Content-Type'].split('/')[-1]
    filename = './images/{}-{}.{}'.format(name, index, extension)
    if not extension in ['jpeg', 'png']:
        # サポートされてない画像拡張子
        return
    with open(filename, 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response

In [5]:
# プログレスバー
from tqdm import tqdm_notebook as tqdm

# 画像保存用のディレクトリを作成
if not os.path.exists('./images'):
    os.mkdir('./images')
    
count = 5
bar = tqdm(total=len(talents))

# それぞれのタレントに対してBingで検索、画像を保存する
for idx, talent in enumerate(talents):
    bar.update(idx+1)
    name_kanji = talent['name_kanji']
    files = glob.glob('./images/{}-{}.*'.format(name_kanji, count-1))
    # すでにダウンロード済みの場合は無視する
    if len(files) == 0:
        results = search_image(name_kanji, count=count)
        for idx, result in enumerate(results):
            image_url = result['contentUrl']
            save_image(image_url, name_kanji, idx)

## 人物情報をCSVに保存

In [59]:
import csv

with open('./talents.csv', 'w') as f:
    headers = list(talents[0].keys())
    writer = csv.writer(f)
    writer.writerow(headers)
    for talent in talents:
        writer.writerow(list(talent.values()))